In [90]:
import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

import pandas as pd
import numpy as np

In [91]:
df = pd.read_csv('https://raw.githubusercontent.com/artist0123/ProjectPSIT/main/master.csv', engine = 'python', sep = ',')

In [92]:
df_country = df.groupby(['Country', 'year']).sum().reset_index()
df_country.replace(0, 1, inplace=True)

In [93]:
code_df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df_country_code = df_country.merge(code_df, left_on='Country', right_on='COUNTRY', how='left')

In [94]:
# Build App
YEARS = [x for x in range(1987, 2015)]
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div(
    id="root",
    children=[
        html.Div(
            id="header",
            className='jumbotron',
            children=[
                html.H1(children="Data Visualization Dashboard")
            ]),
        dbc.Container([#Container
            dbc.Row([#Row
                dbc.Col([#Col1
                        dcc.Slider(
                            id='year-slider',
                            min=min(YEARS),
                            max=max(YEARS),
                            value=min(YEARS),
                        ),
                        dcc.Graph(id='world-choropleth'),
                    ],
                    md=8),#Col1
                dbc.Col([#Col2
                        dcc.Dropdown(
                            id='select-dropdown',
                            options=[
                                {'label': 'New York City', 'value': 'NYC'},
                                {'label': 'Montreal', 'value': 'MTL'},
                                {'label': 'San Francisco', 'value': 'SF'}
                                ],
                            value='NYC'
                            ),
                    html.Div(id='dd-output-container')
                ])#Col2
        ])#Row
    ])#Container
])#Div1

In [95]:
# Define callback to update graph
@app.callback(
    Output(component_id='world-choropleth', component_property='figure'),
    Output(component_id='dd-output-container', component_property='children'),
    [Input(component_id='year-slider', component_property='value'),
    Input(component_id='select-dropdown', component_property='value')]
)
def update_figure(year, pltselect):
    df_choropleth = df_country_code.query('year=='+str(year))
    world_suicide = go.Figure(data=go.Choropleth(
    locations = df_choropleth['CODE'],
    z = df_choropleth['suicides_no'],
    text = df_choropleth['Country'],
    colorscale = 'Oryel',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.2,
    colorbar_title = 'Suicide Cases',
    ))

    world_suicide.update_layout(
    autosize=True,
    title_text='Global Suicide Cases in ' + str(year),
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
        )
    )

    dd_output = 'You have selected' + str(pltselect)
    return world_suicide, dd_output

In [96]:
# Run app and display result inline in the notebook
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
